In [1]:
import socket
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [62]:
class GAViewerClient(object):
    def __init__(self, host, port):
        self._host = host
        self._port = port
        self._socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self._socket.connect((self._host, self._port))
        self._state = self._socket.recv(1024)
        
    def show_multivector(self, multivector, name='ans', color='red', versor=False,
                         stipple=False, wireframe=False, outline=False,
                         weight=True, orientation=False):
        ret = multivector
        
        if versor:
            ret = 'versor({})'.format(ret)
        if stipple:
            ret = 'stipple({})'.format(ret)
        if wireframe:
            ret = 'wireframe({})'.format(ret)
        if outline:
            ret = 'outline({})'.format(ret)
        if not weight:
            ret = 'no_weight({})'.format(ret)
        elif weight:
            ret = 'weight({})'.format(ret)
        if not orientation:
            ret = 'no_ori({})'.format(ret)
        elif orientation:
            ret = 'ori({})'.format(ret)
            
        if type(color) == str:
            ret = '{color}({ret})'.format(color=color, ret=ret)
        elif type(color) == tuple:
            ret = 'color({ret}, {r}, {g}, {b})'.format(ret=ret, r=color[0], g=color[1], b=color[2])
        
        ret = '{name} = '.format(name=name) + ret
                
        return self._send(ret)
    
    def reset(self):
        self.evaluate('reset()')
            
    def evaluate(self, expression):
        return self._send(expression, '')

    def vector(self, vector):
        return '({} e1 + {} e2 + {} e3)'.format(*vector)
    
    def bivector(self, bivector):
        return '({} e1^e2 + {} e1^e3 + {} e2^e3)'.format(*bivector)
    
    def point(self, point):
        return 'c3ga_point({},{},{})'.format(*point)
        
    def rotor(self, rotor):
        return '({} + {} e1^e2 + {} e1^e3 + {} e2^e3)'.format(*rotor)
    
    def translator(self, translator):
        return '(1 - 0.5 ({} e1^ni + {} e2^ni + {} e3^ni))'.format(*translator)
    
    def general_rotor(self, general_rotor):
        return '({} + {} e1^e2 + {} e1^e3 + {} e2^e3 + {} e1^ni + {} e2^ni + {} e3^ni)'.format(*general_rotor)
        
    def _send(self, data, terminator=','):
        self._socket.send('{data}{terminator}$'.format(data=data, terminator=terminator))

In [63]:
gavcl = GAViewerClient('localhost', 6882)

In [75]:
gr = gavcl.general_rotor([1.,0.0,0.0,0.0,0.0,0.0,0.0,3.0])
gavcl.show_multivector(gr, 'G', versor=True)
gavcl.show_multivector(gavcl.point([1.,0.,0.]), 'p')
gavcl.evaluate('b = vp(G, p),')

In [47]:
program = [
    'batch init();',
    'batch vtrail();',
    'set_window_title("Chasles\'s screw");',
    'init(0);',
    'campos(20 e3)',
    'set_line_width(4);'
    'ctrl_range(phi, 0, pi, 0.01);',
    'phi = 0.7;',
    'dynamic{R: R = exp(e1^e2 phi/2),};',
    'dynamic{I: I=grade(R,2); I=I/norm(I);};',
    'tvec = e1+e2- e3,',
    'dynamic{t: t= c3ga_point(tvec);};',
    'dynamic{w: w = (tvec^I)/I,}; label(w,"\eqn{\bold{w}}");',
    'dynamic{v: v = 1/(1-R R)  (tvec.I)/I,}; label(v,"\eqn{\bold{v}}");',
    'dynamic{orgV: orgV = tv(tvec) R, };',
    'dynamic{simV: simV = tv(w) vp(tv(v),R),};',
    'A = vp(tv(2 e3),no^e2^e3);',
    'dynamic{tanB: tanB= color(vp(tv(v), no^I),1,1,0,0.9),};',
    'dynamic{simtrail: vtrail(simV,A),};',
    'dynamic{orgtrail: vtrail(orgV,A),};',
    'dynamic{lineaxis: lineaxis = black(vp(tv(v),no^w^ni)),};',
    'dynamic{RvR: RvR =alpha(R v /R, 0.2),}; label(RvR, "\eqn{R \bold{v}/R");',
    'dynamic{vR: vR =dm2(c3ga_point(v)^c3ga_point(RvR)),};',
    'dynamic{wR: wR =dm2(c3ga_point(t)^c3ga_point(w)),};',
    'dynamic{wt: wt= alpha(dm3(factored_bivector(w,(t-w))),0.3),};',
    'dynamic{vRvR: vRvR= alpha(dm3(factored_bivector(RvR,(t-w))),0.3),};',
    'set_point_size(0);',
    'camori = 4.3e-001 + -4.7e-001*e2^e3 + 6.7e-001*e3^e1 + 3.5e-001*e1^e2;',
]
for line in program:
    gavcl.evaluate(line)
    






In [15]:
gavcl.show_multivector(gavcl.vector([1., 1., 1.]), 'a', (1.,1.,0.))

'"a" = 1.000000000000000e+00*e1 + 1.000000000000000e+00*e2 + 1.000000000000000e+00*e3,$'

In [ ]:
gavcl.show_multivector(gavcl.point([1., 1., 0. ]), 'p', 'cyan')

In [ ]:
gavcl.evaluate('p ^ a ^ ni')
L1 = gavcl.show_multivector('f[] ^ a ^ ni', 'L1', (1.0, 0.0, 0.0))
print(L1)

In [ ]:
import time
j = 0
for i in np.linspace(0,np.pi):
    p = [cos(i), sin(i), 0]
    gavcl.show_multivector(gavcl.point(p), 'f[{}]'.format(j), 'yellow')
    gavcl.evaluate('L[{}] = f[{}] ^ ({}) ^ ni'.format(j,j, gavcl.vector(p)))
    j += 1
#     time.sleep(0.01)

In [ ]:
gavcl.show_multivector(gavcl.rotor([cos(pi/6), -sin(pi/6), 0., 0. ]), 'R', 'red', versor=True)

In [ ]:
_ = gavcl.show_multivector(gavcl.translator([1.,0.,0.]) + ' ' + gavcl.rotor([cos(pi/6), -sin(pi/6), 0., 0. ]), 'M', 'red', versor=True)

In [ ]:
gavcl.reset()